**Zipfile von EFFIS herunterladen und entpacken**

zwei **Regionen**
zwei Spalten
np Befehl
-> mehr als ein Treffer, ein Polygon
-> zweite, dritte Spalte
-> falls Ha fehlen, nicht selbst berechnen (müsste umgerechnet werden)

In [1]:
import requests
import zipfile
import os
import geopandas as gpd
import pandas as pd

In [2]:
# URL des Shapefiles von EFFIS(European Forest Fire Information System)

url = "https://maps.effis.emergency.copernicus.eu/effis?service=WFS&request=getfeature&typename=ms:modis.ba.poly&version=1.1.0&outputformat=SHAPEZIP"

In [3]:
response = requests.get(url)
# checken, ob Get-Request erfolgreich war
if response.status_code == 200:
  print("Request erfolgreich")
  # zip Datei speichern
  zip_file = "waldbrände.zip"
# Inhalt des Get-Requests in Datei speichern
  with open(zip_file, "wb") as file:
    file.write(response.content)

print("Datei heruntergeladen und gespeichert")

Request erfolgreich
Datei heruntergeladen und gespeichert


In [4]:
# Notlösung, falls Download über Effis nicht möglich ist
# ansonsten nicht ausführen
#zip_file = "/content/drive/MyDrive/Waldbrände/effis_layer (2).zip"

In [5]:
response

<Response [200]>

In [6]:
# Zipfile entpacken
with zipfile.ZipFile(zip_file, "r") as zip_ref:
  zip_ref.extractall("waldbrände_shapefile")

print("Zip Datei entpackt")

Zip Datei entpackt


In [7]:
# Verzeichis durchsuchen und Pfade zu allen Dateien ausgeben
verzeichnis = "waldbrände_shapefile"
for root, dirs, files in os.walk(verzeichnis):
  for name in files:
    print(os.path.join(root, name))

waldbrände_shapefile/modis.ba.poly.prj
waldbrände_shapefile/modis.ba.poly.shp
waldbrände_shapefile/modis.ba.poly.shx
waldbrände_shapefile/modis.ba.poly.dbf


In [8]:
# Shapefile Pfad in Variable speichern

shapefile = os.path.join(verzeichnis, "/content/waldbrände_shapefile/modis.ba.poly.shp") # Pfad muss ggf. angepasst werden

In [9]:
# prüfen, ob Pfad zum Shapefile passt
if not os.path.exists(shapefile):
  print("Shapefile nicht gefunden")
else:
  # Shapefile mit Geopandas öffnen und als Geodataframe speichern
  waldbrände_gdf = gpd.read_file(shapefile)

In [10]:
waldbrände_gdf.head()

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,SCLEROPH,TRANSIT,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry
0,27361,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Foggia,San Giovanni Rotondo,95,6.24999999999349,1.0416666666655816,0,0,0,92.70833333323677,0,0,0,99.99999999999999,FireSeason,"POLYGON ((15.71638 41.72314, 15.71550 41.71508..."
1,27785,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,0,0,0,0,79.77528089878678,0,20.22471910110087,0,0,100,FireSeason,"POLYGON ((-7.83344 41.05790, -7.83091 41.05666..."
2,27809,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Ragusa,Ragusa,126,0,0,0,0,0,0,90.47619047611867,9.523809523801965,0,0,FireSeason,"POLYGON ((14.69460 36.90994, 14.69246 36.90844..."
3,27827,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,0,0,0,0,18.632075471693717,80.42452830186782,0.9433962264148719,0,0,0,FireSeason,"POLYGON ((-8.17826 41.14283, -8.17150 41.14231..."
4,27865,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,0,0,0,0,19.117647058795413,76.47058823518165,4.411764705875864,0,0,0,FireSeason,"POLYGON ((-7.78673 40.50095, -7.79049 40.49869..."


In [11]:
# Checken welches Koordinaten Referenzsystem die Geodaten haben
geodaten = waldbrände_gdf["geometry"]
print(geodaten.crs)

GEOGCS["GCS_unknown",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433],AXIS["Longitude",EAST],AXIS["Latitude",NORTH]]


In [12]:
# Koordinaten-Referenzsystem zu EPSG:4326 ändern

waldbrände_gdf["geometry"] = waldbrände_gdf["geometry"].to_crs(epsg=4326)
print(waldbrände_gdf["geometry"].crs)

EPSG:4326


**Überprüfen mit Qgis**

In [13]:
# Erstelle Zipfile mit Shapefile, um zu überprüfen, ob es mit den geupdtateten Koordinatenreferenzsystem passt
# Zipfile bei Qgis reinziehen, ursprüngliches Shapefile von EFFIS auch reinziehen -> stimmen überein

import shutil
import os

output_folder = "output_shapefile"
output_shapefile = f"{output_folder}/my_shapefile.shp"
output_zipfile = "my_shapefile.zip"

os.makedirs(output_folder, exist_ok=True)


waldbrände_gdf.to_file(output_shapefile)

shutil.make_archive(output_folder, "zip", output_folder)

os.rename(f"{output_folder}.zip", output_zipfile)

shutil.rmtree(output_folder)

**Cleanen usw.**

In [14]:
waldbrände_gdf["FIREDATE"].describe()

,FIREDATE
count,80173
unique,22702
top,2001-01-01 00:00:00
freq,1034


In [15]:
# Datum-Spalte in Datetime umwandeln und Jahr-Spalte hinzufügen
waldbrände_gdf["FIREDATE"] = pd.to_datetime(waldbrände_gdf["FIREDATE"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
waldbrände_gdf["Year"] = waldbrände_gdf["FIREDATE"].dt.year

In [16]:
waldbrände_gdf["COUNTRY"].describe()

,COUNTRY
count,80165
unique,48
top,IT
freq,11135


In [17]:
waldbrände_gdf["PROVINCE"].value_counts()

,count
PROVINCE,
N.A.,17207
Asturias,2225
Pyrénées-Atlantiques,1773
???? ????,1669
Cantabria,1365
...,...
Lecco,1
Borore,1
Kfar Chlane,1


In [18]:
waldbrände_gdf["COMMUNE"].describe()

,COMMUNE
count,80173
unique,9075
top,N.A.
freq,28451


In [19]:
# NaNs in der Länderspalte genauer anschauen
NaN_Länder = waldbrände_gdf[waldbrände_gdf["COUNTRY"].isna()]
NaN_Länder

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,SCLEROPH,TRANSIT,OTHERNATLC,AGRIAREAS,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry,Year
5648,215943,2003-05-01 00:00:00,2003-05-01 00:00:00,None,N.A.,N.A.,31,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((35.18694 36.68751, 35.18652 36.68739...",2003.0
5872,215944,2003-05-25 00:00:00,2003-05-25 00:00:00,None,N.A.,N.A.,19,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((-2.97404 54.14677, -2.97404 54.14677...",2003.0
5873,215946,2003-05-04 00:00:00,2003-05-04 00:00:00,None,N.A.,N.A.,103,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((35.55811 36.69824, 35.54932 36.69824...",2003.0
5879,215947,2003-11-06 00:00:00,2003-11-06 00:00:00,None,N.A.,N.A.,31,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((35.18694 36.68751, 35.18652 36.68739...",2003.0
5892,215948,2003-05-31 00:00:00,2003-05-31 00:00:00,None,N.A.,N.A.,27,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((-2.99756 53.67241, -2.99667 53.67265...",2003.0
7974,215949,2004-06-01 00:00:00,2004-06-06 00:00:00,None,N.A.,N.A.,148,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((35.57522 36.69463, 35.57564 36.69448...",2004.0
70703,227055,2024-03-09 11:04:00,2024-03-09 11:04:00,None,N.A.,N.A.,8,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((5.10658 60.09812, 5.10665 60.09832, ...",2024.0
70706,227060,2024-03-10 10:43:00,2024-03-10 10:43:00,None,N.A.,N.A.,6,None,None,None,None,None,None,None,None,None,0,FireSeason,"POLYGON ((4.82455 61.15870, 4.82512 61.15851, ...",2024.0


In [21]:
# neue Karte mit Ländergrenzen -> Link: https://www.naturalearthdata.com/downloads/10m-cultural-vectors/ (erster Download auf der Seite)

zip_file_weltkarte = "/content/drive/MyDrive/Waldbrände/ne_10m_admin_0_countries.zip"

pfad_weltkarte = "weltkarte_ländergrenzen"

with zipfile.ZipFile(zip_file_weltkarte, "r") as zip_ref:
  zip_ref.extractall(pfad_weltkarte)
  print(f"ZIP-Datei erfolgreich extrahiert")

  for file_name in zip_ref.namelist():
    print(f"Extrahierte Datei: {file_name}")

ZIP-Datei erfolgreich extrahiert
Extrahierte Datei: ne_10m_admin_0_countries.README.html
Extrahierte Datei: ne_10m_admin_0_countries.VERSION.txt
Extrahierte Datei: ne_10m_admin_0_countries.cpg
Extrahierte Datei: ne_10m_admin_0_countries.dbf
Extrahierte Datei: ne_10m_admin_0_countries.prj
Extrahierte Datei: ne_10m_admin_0_countries.shp
Extrahierte Datei: ne_10m_admin_0_countries.shx


In [22]:
# Shapefile Pfad in Variable speichern
weltkarte = gpd.read_file("/content/weltkarte_ländergrenzen/ne_10m_admin_0_countries.shp") # Pfad muss ggf. angepasst werden

In [23]:
weltkarte.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,TLC,ADMIN,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,0,2,Indonesia,IDN,0,2,Sovereign country,1,Indonesia,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Admin-0 country,0,3,Malaysia,MYS,0,2,Sovereign country,1,Malaysia,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Admin-0 country,0,2,Chile,CHL,0,2,Sovereign country,1,Chile,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Admin-0 country,0,3,Bolivia,BOL,0,2,Sovereign country,1,Bolivia,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Admin-0 country,0,2,Peru,PER,0,2,Sovereign country,1,Peru,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [24]:
weltkarte.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 258 entries, 0 to 257
Columns: 169 entries, featurecla to geometry
dtypes: float64(6), geometry(1), int64(25), object(137)
memory usage: 340.8+ KB


In [25]:
# hier noch alles auskommentieren

In [26]:
# Geodaten zu Ländern zuordnen (dauert sehr lange)
waldbrände_gdf_ländergrenzen = gpd.sjoin(waldbrände_gdf, weltkarte, how="left", op="intersects")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [27]:
länderliste = waldbrände_gdf_ländergrenzen.groupby(waldbrände_gdf_ländergrenzen.index)["NAME_DE"].apply(list)

In [28]:
waldbrände_gdf_ländergrenzen["betroffene_länder"] = länderliste

In [29]:
waldbrände_gdf_ländergrenzen = waldbrände_gdf_ländergrenzen.loc[~waldbrände_gdf_ländergrenzen.index.duplicated(keep='first')]

In [30]:
waldbrände_gdf_ländergrenzen

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,betroffene_länder
0,27361,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Foggia,San Giovanni Rotondo,95,6.24999999999349,1.0416666666655816,0,...,None,None,None,None,None,None,None,None,None,[Italien]
1,27785,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,0,0,0,...,None,None,None,None,None,None,None,None,None,[Portugal]
2,27809,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Ragusa,Ragusa,126,0,0,0,...,None,None,None,None,None,None,None,None,None,[Italien]
3,27827,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,0,0,0,...,None,None,None,None,None,None,None,None,None,[Portugal]
4,27865,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,0,0,0,...,None,None,None,None,None,None,None,None,None,[Portugal]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80168,239167,2024-08-17 00:30:00,2024-08-18 01:02:00,BA,Fo?a,N.A.,1,None,None,None,...,None,None,None,None,None,None,None,None,None,[Bosnien und Herzegowina]
80169,239170,2024-08-14 00:11:00,2024-08-17 00:30:00,ME,???? ????,N.A.,3,59.9999999988,0,39.9999999992,...,None,None,None,None,None,None,None,None,None,[Montenegro]
80170,238275,2024-08-19 01:37:00,2024-08-19 03:15:00,ES,Sevilla,Alcalá del Río,3,0,0,0,...,None,None,None,None,None,None,None,None,None,[Spanien]
80171,239169,2024-08-13 00:05:00,2024-08-17 00:30:00,ME,???? ????,N.A.,30,6.89655172411415,37.93103448262783,13.7931034482283,...,None,None,None,None,None,None,None,None,None,[Montenegro]


In [31]:
# neue Karte mit Provinzgrenzen -> Link: https://www.naturalearthdata.com/downloads/10m-cultural-vectors/ (Admin 1 - States, Provinces)

zip_file_provinzen = "/content/drive/MyDrive/Waldbrände/ne_10m_admin_1_states_provinces.zip"

pfad_provinzen = "provinzen"

with zipfile.ZipFile(zip_file_provinzen, "r") as zip_ref:
  zip_ref.extractall(pfad_provinzen)
  print(f"ZIP-Datei erfolgreich extrahiert")

  for file_name in zip_ref.namelist():
    print(f"Extrahierte Datei: {file_name}")

ZIP-Datei erfolgreich extrahiert
Extrahierte Datei: ne_10m_admin_1_states_provinces.README.html
Extrahierte Datei: ne_10m_admin_1_states_provinces.VERSION.txt
Extrahierte Datei: ne_10m_admin_1_states_provinces.cpg
Extrahierte Datei: ne_10m_admin_1_states_provinces.dbf
Extrahierte Datei: ne_10m_admin_1_states_provinces.prj
Extrahierte Datei: ne_10m_admin_1_states_provinces.shp
Extrahierte Datei: ne_10m_admin_1_states_provinces.shx


In [32]:
# Shapefile Pfad in Variable speichern
provinzen = gpd.read_file("/content/provinzen/ne_10m_admin_1_states_provinces.shp") # Pfad muss ggf. angepasst werden

In [33]:
# Provinzen/Gemeinden zu Geodataframe hinzufügen
waldbrände_provinzen_ländergrenzen = gpd.sjoin(waldbrände_gdf_ländergrenzen, provinzen, how="left", op="intersects", lsuffix="_left", rsuffix="_right")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [34]:
# Liste mit Provinzen für jeden Brand erstellen
provinzliste = waldbrände_provinzen_ländergrenzen.groupby(waldbrände_provinzen_ländergrenzen.index)["name_de"].apply(list)

In [35]:
waldbrände_provinzen_ländergrenzen["betroffene_gemeinden"] = provinzliste

In [36]:
waldbrände_provinzen_ländergrenzen = waldbrände_provinzen_ländergrenzen.loc[~waldbrände_provinzen_ländergrenzen.index.duplicated(keep='first')]

In [37]:
waldbrände_provinzen_ländergrenzen

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,FCLASS_ID__right,FCLASS_PL__right,FCLASS_GR__right,FCLASS_IT__right,FCLASS_NL__right,FCLASS_SE__right,FCLASS_BD__right,FCLASS_UA__right,FCLASS_TLC__right,betroffene_gemeinden
0,27361,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Foggia,San Giovanni Rotondo,95,6.24999999999349,1.0416666666655816,0,...,None,None,None,None,None,None,None,None,None,[Foggia]
1,27785,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,0,0,0,...,None,None,None,None,None,None,None,None,None,[Viseu]
2,27809,2000-01-01 00:00:00,2000-01-01 00:00:00,IT,Ragusa,Ragusa,126,0,0,0,...,None,None,None,None,None,None,None,None,None,[Ragusa]
3,27827,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,0,0,0,...,None,None,None,None,None,None,None,None,None,[Porto]
4,27865,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,0,0,0,...,None,None,None,None,None,None,None,None,None,[Guarda]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80168,239167,2024-08-17 00:30:00,2024-08-18 01:02:00,BA,Fo?a,N.A.,1,None,None,None,...,None,None,None,None,None,None,None,None,None,[Foča]
80169,239170,2024-08-14 00:11:00,2024-08-17 00:30:00,ME,???? ????,N.A.,3,59.9999999988,0,39.9999999992,...,None,None,None,None,None,None,None,None,None,[Berane]
80170,238275,2024-08-19 01:37:00,2024-08-19 03:15:00,ES,Sevilla,Alcalá del Río,3,0,0,0,...,None,None,None,None,None,None,None,None,None,[Sevilla]
80171,239169,2024-08-13 00:05:00,2024-08-17 00:30:00,ME,???? ????,N.A.,30,6.89655172411415,37.93103448262783,13.7931034482283,...,None,None,None,None,None,None,None,None,None,[Berane]


In [39]:
# unnötige Zeilen löschen
waldbrände_mehrere_länder_u_provinzen = waldbrände_provinzen_ländergrenzen[['FIREDATE','LASTUPDATE',"COUNTRY","PROVINCE","COMMUNE","AREA_HA","geometry","Year","NAME_DE","name_de","betroffene_länder","betroffene_gemeinden"]]

In [40]:
waldbrände_mehrere_länder_u_provinzen.head()

,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,geometry,Year,NAME_DE,name_de,betroffene_länder,betroffene_gemeinden
0,2000-01-01,2000-01-01 00:00:00,IT,Foggia,San Giovanni Rotondo,95,"POLYGON ((15.71638 41.72314, 15.71550 41.71508...",2000.0,Italien,Foggia,[Italien],[Foggia]
1,2000-01-01,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,"POLYGON ((-7.83344 41.05790, -7.83091 41.05666...",2000.0,Portugal,Viseu,[Portugal],[Viseu]
2,2000-01-01,2000-01-01 00:00:00,IT,Ragusa,Ragusa,126,"POLYGON ((14.69460 36.90994, 14.69246 36.90844...",2000.0,Italien,Ragusa,[Italien],[Ragusa]
3,2000-01-01,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,"POLYGON ((-8.17826 41.14283, -8.17150 41.14231...",2000.0,Portugal,Porto,[Portugal],[Porto]
4,2000-01-01,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,"POLYGON ((-7.78673 40.50095, -7.79049 40.49869...",2000.0,Portugal,Guarda,[Portugal],[Guarda]


In [41]:
waldbrände_mehrere_länder_u_provinzen["betroffene_länder"].value_counts()

,count
betroffene_länder,
[Italien],11119
[Spanien],9865
[Portugal],9233
[Ukraine],9069
[Türkei],4963
...,...
[UN-Pufferzone in Zypern],1
[Georgien],1
"[Kroatien, Ungarn]",1


In [44]:
# überprüfen, ob die Länder- und Gemeindelisten mindestens einen Wert enthalten
leere_listen = waldbrände_mehrere_länder_u_provinzen["betroffene_gemeinden"].apply(lambda lst: len(lst)>0)
überprüfung = leere_listen.all()
print(überprüfung)

In [46]:
# überprüfen, ob die exakt selben Geodaten mehrmals auftauchen
waldbrände_mehrere_länder_u_provinzen["geometry"].value_counts()

,count
geometry,
"POLYGON ((1.11572 35.92920, 1.11133 35.92920, ...",2
"POLYGON ((35.18694 36.68751, 35.18652 36.68739...",2
"POLYGON ((-7.32178 42.21777, -7.32617 42.21777...",2
"POLYGON ((24.80225 43.21533, 24.79785 43.21533...",2
"POLYGON ((27.08740 44.41943, 27.08301 44.41943...",2
...,...
"POLYGON ((7.23637 36.44490, 7.23644 36.44488, ...",1
"POLYGON ((6.24400 36.68320, 6.24408 36.68314, ...",1
"POLYGON ((7.28262 36.89446, 7.28288 36.89453, ...",1


In [ ]:
filtered_df

**Next steps:**

*   wo kommen die Unterschiede bei Provinzen zwischen EFFIS-Daten und meinen Daten her?
*   Enthalten alle Einträge in betroffene_länder und betroffene_gemeinden mindestens einen Ort




In [47]:
# Kopie erstellen, um Zeilen mit denselben Geodaten genauer zu untersuchen
duplikate_suche = waldbrände_mehrere_länder_u_provinzen.copy()

In [48]:
# Zeilen mit denselben Geodaten UND demselben Datum suchen
duplikate_suche["duplikate_count"] = duplikate_suche.groupby(["geometry","FIREDATE"])["geometry"].transform("count")


# Duplikate in neuem Geodataframe speichern
duplikate = duplikate_suche[duplikate_suche["duplikate_count"] > 1].drop(columns="duplikate_count")

In [49]:
# Duplikate anschauen
duplikate

,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,geometry,Year,NAME_DE,name_de,betroffene_länder,betroffene_gemeinden


In [ ]:
# keine Waldbrände mit selbem Datum UND exakt selben Geodaten mehr

**Daten für einzelne Länder**

In [56]:
land = "Portugal" # hier Land eintragen, für das man sich interessiert

waldbrände_filter = waldbrände_mehrere_länder_u_provinzen[waldbrände_mehrere_länder_u_provinzen["betroffene_länder"].apply(lambda x: land in x)]

waldbrände_filter

,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,geometry,Year,NAME_DE,name_de,betroffene_länder,betroffene_gemeinden
1,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Douro,"Cepões, Meijinhos e Melcões",91,"POLYGON ((-7.83344 41.05790, -7.83091 41.05666...",2000.0,Portugal,Viseu,[Portugal],[Viseu]
3,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Tâmega e Sousa,Avessadas e Rosém,432,"POLYGON ((-8.17826 41.14283, -8.17150 41.14231...",2000.0,Portugal,Porto,[Portugal],[Porto]
4,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Paranhos,70,"POLYGON ((-7.78673 40.50095, -7.79049 40.49869...",2000.0,Portugal,Guarda,[Portugal],[Guarda]
10,2000-01-01 00:00:00,2000-01-01 00:00:00,ES,Pontevedra,Arbo,46,"POLYGON ((-8.28205 42.13048, -8.28309 42.12701...",2000.0,Spanien,Viana do Castelo,"[Spanien, Portugal]","[Viana do Castelo, Pontevedra]"
11,2000-01-01 00:00:00,2000-01-01 00:00:00,PT,Beiras e Serra da Estrela,Caria,55,"POLYGON ((-7.29869 40.26370, -7.30037 40.26179...",2000.0,Portugal,Castelo Branco,[Portugal],[Castelo Branco]
...,...,...,...,...,...,...,...,...,...,...,...,...
79962,2024-08-21 13:59:00,2024-08-22 03:08:00,PT,Tâmega e Sousa,Valpedre,0,"POLYGON ((-8.33968 41.14172, -8.33955 41.14184...",2024.0,Portugal,Porto,[Portugal],[Porto]
79963,2024-08-21 13:10:00,2024-08-21 13:10:00,PT,Tâmega e Sousa,"Silvares, Pias, Nogueira e Alvarenga",8,"POLYGON ((-8.27277 41.29573, -8.27251 41.29563...",2024.0,Portugal,Porto,[Portugal],[Porto]
79964,2024-08-21 01:22:00,2024-08-21 12:46:00,PT,Alto Minho,Gandra e Taião,383,"POLYGON ((-8.57692 41.99595, -8.57624 41.99611...",2024.0,Portugal,Viana do Castelo,[Portugal],[Viana do Castelo]
79965,NaT,2024-08-17 10:00:41.067,PT,Alto Minho,Portela e Extremo,1,"POLYGON ((-8.45795 41.94758, -8.45787 41.94760...",NaN,Portugal,Viana do Castelo,[Portugal],[Viana do Castelo]


In [60]:
waldbrände_filter_nach_jahr["Jahr"].value_counts()

,count
Jahr,
2022.0,905
2023.0,661
2005.0,591
2001.0,583
2021.0,552
2002.0,551
2003.0,453
2017.0,415
2020.0,383


In [68]:
from datetime import datetime
aktuelles_datum = datetime.now()
aktuelles_jahr = aktuelles_datum.year

# Daten aus dem Jahr 2000 müssen ausgeschlossen werden, weil für sie keine Monate und Tage vorliegen
waldbrände_filter_nach_jahr = waldbrände_filter[waldbrände_filter["FIREDATE"].dt.year != 2000].copy()

# Spalte mit Jahr hinzufügen
waldbrände_filter_nach_jahr["Jahr"] = waldbrände_filter_nach_jahr["FIREDATE"].dt.year

# Durchschnittliche Zahl der Waldbrände pro Jahr bis zum aktuellen Jahr und Datum ausrechnen
def waldbrände_pro_jahr(df, aktuelles_datum):
    maske = (df["FIREDATE"].dt.month < aktuelles_datum.month) |\
            ((df["FIREDATE"].dt.month == aktuelles_datum.month) & (df["FIREDATE"].dt.day <= aktuelles_datum.day))
    return df[mask].shape[0]

jahre = waldbrände_filter_nach_jahr["FIREDATE"].dt.year.unique()
durchschnittliche_waldbrände_pro_jahr = {
    jahr: waldbrände_pro_jahr(waldbrände_filter_nach_jahr[waldbrände_filter_nach_jahr["FIREDATE"].dt.year == jahr, aktuelles_datum])
    for jahr in jahre if jahr != aktuelles_jahr
}


# Anzahl der Waldbrände im aktuellen Jahr berechnen
Waldbrände_2024 = waldbrände_filter_nach_jahr[waldbrände_filter_nach_jahr["Jahr"]== aktuelles_jahr]
Waldbrände_2024_anzahl = Waldbrände_2024[
    (Waldbrände_2024["FIREDATE"].dt.month < aktuelles_datum.month) |
    ((Waldbrände_2024["FIREDATE"].dt.month == aktuelles_datum.month) & (Waldbrände_2024["FIREDATE"].dt.day <= aktuelles_datum.day))
].shape[0]


# Vergleich
if Waldbrände_2024_anzahl > durchschnittliche_waldbrände_pro_jahr:
    print(f"In ({land}) gab im aktuellen Jahr ({aktuelles_jahr}) ungewöhnlich viele Waldbrände: {Waldbrände_2024_anzahl} (Durchschnitt: {durchschnittliche_waldbrände_pro_jahr:.2f})")
else:
    print(f"Die Anzahl der Waldbrände im aktuellen Jahr ({aktuelles_jahr}) liegt unter dem Durchschnitt: {Waldbrände_2024_anzahl} (Durchschnitt: {durchschnittliche_waldbrände_pro_jahr:.2f})")

InvalidIndexError: (793       True
796       True
797       True
799       True
803       True
         ...  
79962    False
79963    False
79964    False
79965    False
80083    False
Name: FIREDATE, Length: 9252, dtype: bool, datetime.datetime(2024, 8, 26, 9, 58, 40, 927348))

In [ ]:
# überlappende Polygone?
# mapshaper?
#

In [ ]:
durchschnittliche_waldbrände_pro_jahr = waldbrände_filter_nach_jahr[waldbrände_filter_nach_jahr["FIREDATE"].dt.year != aktuelles_jahr].groupby("FIREDATE").apply(
    lambda x: x[x["FIREDATE"].dt.month < aktuelles_datum.month].shape[0] |
              x[(x["FIREDATE"].dt.month == aktuelles_datum.month) & (x["FIREDATE"].dt.day <= aktuelles_datum.day)].shape[0]
).mean()


**Daten für einzelne Bundesländer**

In [53]:
bundesland = "Baden-Württemberg" # hier Bundesland/Gemeinde eintragen, für das man sich interessiert

waldbrände_filter_gemeinde = waldbrände_mehrere_länder_u_provinzen[waldbrände_mehrere_länder_u_provinzen["betroffene_gemeinden"].apply(lambda x: bundesland in x)]

waldbrände_filter_gemeinde

,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,geometry,Year,NAME_DE,name_de,betroffene_länder,betroffene_gemeinden
3119,2001-08-31 00:00:00,2001-08-31 00:00:00,DE,Neckar-Odenwald-Kreis,Limbach,16,"POLYGON ((9.12695 49.48633, 9.13135 49.48633, ...",2001.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
6622,2003-02-19 00:00:00,2003-03-05 00:00:00,DE,Schwäbisch Hall,Fichtenberg,1431,"POLYGON ((9.71582 48.96338, 9.71582 48.95898, ...",2003.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
6628,2003-02-22 00:00:00,2003-02-22 00:00:00,DE,Rems-Murr-Kreis,"Murrhardt, Stadt",31,"POLYGON ((9.65869 48.96777, 9.66748 48.96777, ...",2003.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
6842,2003-07-21 00:00:00,2003-07-21 00:00:00,DE,Calw,Enzklösterle,16,"POLYGON ((8.44580 48.68652, 8.45020 48.68652, ...",2003.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
8451,2004-07-14 00:00:00,2004-07-14 00:00:00,DE,"Karlsruhe, Landkreis",Gondelsheim,16,"POLYGON ((8.65234 49.04248, 8.65234 49.04687, ...",2004.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
8533,2004-07-24 00:00:00,2004-07-29 00:00:00,DE,"Karlsruhe, Landkreis","Bretten, Stadt",251,"POLYGON ((8.66992 49.04687, 8.66553 49.04687, ...",2004.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
10755,2005-06-26 00:00:00,2005-06-26 00:00:00,DE,Calw,Oberreichenbach,16,"POLYGON ((8.65234 48.75684, 8.64795 48.75684, ...",2005.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
50052,2022-03-22 12:02:00,2022-03-22 12:02:00,DE,Sigmaringen,Stetten am kalten Markt,9,"POLYGON ((9.06384 48.14097, 9.06392 48.14047, ...",2022.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
50068,2022-03-08 13:05:00,2022-03-08 13:05:00,DE,Zollernalbkreis,"Albstadt, Stadt",14,"POLYGON ((9.02350 48.17892, 9.02389 48.17835, ...",2022.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
78562,2024-07-29 00:35:00,2024-07-29 01:26:00,DE,Schwäbisch Hall,"Langenburg, Stadt",9,"POLYGON ((9.84806 49.22723, 9.84843 49.22716, ...",2024.0,Deutschland,Baden-Württemberg,[Deutschland],[Baden-Württemberg]
